In [1]:
import torch 
from torch.utils.data import Dataset
from dataclasses import dataclass
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import numpy as np

In [ ]:
class CathodeClassifierDataset(Dataset):

    def __init__(self,  generated_dataset, reference_dataset, signal_noise_ratio=0.01, mass_window=[3300, 3700]):

        self.generated_dataset = generated_dataset
        self.reference_dataset = reference_dataset
        self.snr = signal_noise_ratio

        self.get_generated_data()
        self.get_reference_data()

    def __getitem__(self, idx):
        output = {}
        output['background'] = self.background[idx]
        output['data'] = self.source_preprocess[idx]
        return output

    def __len__(self):
        return self.target.size(0)
    
    def __iter__(self):
        for i in range(len(self)):
            yield self[i]

    def get_generated_data(self):
        f = h5py.File(self.dataset, 'r') 
        self.background = torch.Tensor(f['jet high level features'])
        self.
        f.close()

    def get_reference_data(self):
        f = h5py.File(self.dataset, 'r') 
        self.data = torch.Tensor(f['jet features'])
        f.close()




In [ ]:
dataset = CathodeClassifierDataset(generated='../../results/LHCOlympics.SchrodingerBridgeFlowMatching.MLP_forward.2023.11.30_20h51',
                                   reference='../../data/events_anomalydetection.h5',
                                   signal_noise_ratio=0.01,
                                   mass_window=[3300, 3700])